In [1]:
import requests
import pandas
import geopandas
import shapely

# 0. User defined variables

In [2]:
url='http://serveisoberts.gencat.cat/api/equipaments?&exact_matching=&categories.tema=Administraci%C3%B3%20p%C3%BAblica&categories.subtema=Ens%20locals&categories.subtema2=Ajuntaments&adv=true'

# 1. Read data

In [3]:
# not working because:
# DriverError: '/vsimem/b90a75d3a81f4494ac7992a7be08428d' not recognized as a supported file format.

# geopandas.read_file(url)

In [4]:
response=requests.get(url)

In [5]:
data=response.json()

# 2. Manipulate data

## 2.1. Create a smaller dictionary with only relevant fields

In [6]:
relevant_fields=['id','titol','utx','uty','phone']
dictionary={}
for field in relevant_fields:
    dictionary[field]=[]

# iterate over items to generate relevant dictionary
# for item in data['items']:
for i in range(5):
    item=data['items'][i]
    
    dictionary['id'].append(item['id'])
    dictionary['titol'].append(item['titol'])
    dictionary['utx'].append(item['utm']['utmx'])
    dictionary['uty'].append(item['utm']['utmy'])
    dictionary['phone'].append(','.join([element for element in item['telefons'] if len(element) > 3]))
                               
print(dictionary)  

{'id': ['547500', '546860', '547615', '547054', '547243'], 'titol': ['Ajuntament de Sant Jordi Desvalls', 'Ajuntament de Barberà de la Conca', 'Ajuntament de Vilanova de Prades', 'Ajuntament de la Riera de Gaià', 'Ajuntament de Mont-ras'], 'utx': [496105.343, 351852.508, 329001.907, 362557.215, 511997.68], 'uty': [4657722.413, 4585885.731, 4579409.957, 4558359.595, 4639582.972], 'phone': ['972-798005', '977-887010', '977-869056', '977-655000', '972-301974']}


## 2.2. Convert dictionary into pandas dataframe

In [7]:
df=pandas.DataFrame.from_dict(dictionary)
df.head()

,id,titol,utx,uty,phone
0,547500,Ajuntament de Sant Jordi Desvalls,496105.343,4657722.413,972-798005
1,546860,Ajuntament de Barberà de la Conca,351852.508,4585885.731,977-887010
2,547615,Ajuntament de Vilanova de Prades,329001.907,4579409.957,977-869056
3,547054,Ajuntament de la Riera de Gaià,362557.215,4558359.595,977-655000
4,547243,Ajuntament de Mont-ras,511997.680,4639582.972,972-301974


In [8]:
df.to_csv('locations.txt',sep='\t')

## 2.3. Convert pandas dataframe to geopandas dataframe

In [9]:
geometry=[shapely.geometry.Point(xy) for xy in zip(df.utx,df.uty)]
crs={'init':'epsg:25831'}
gdf=geopandas.GeoDataFrame(df,crs=crs,geometry=geometry)
gdf.head()

,id,titol,utx,uty,phone,geometry
0,547500,Ajuntament de Sant Jordi Desvalls,496105.343,4657722.413,972-798005,POINT (496105.343 4657722.413)
1,546860,Ajuntament de Barberà de la Conca,351852.508,4585885.731,977-887010,POINT (351852.508 4585885.731)
2,547615,Ajuntament de Vilanova de Prades,329001.907,4579409.957,977-869056,POINT (329001.907 4579409.957)
3,547054,Ajuntament de la Riera de Gaià,362557.215,4558359.595,977-655000,POINT (362557.215 4558359.595)
4,547243,Ajuntament de Mont-ras,511997.680,4639582.972,972-301974,POINT (511997.68 4639582.972)


# 3. Save to shapefile

In [10]:
gdf.to_file("locations.shp")